In [1]:
!pip install numpy
!pip install wikipedia
!pip install lxml
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [46]:
import lxml
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm 
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import wikipedia as wp

### Installing and importing complete

<br>

#  __PART1__

#### Getting Postal Code from CSV

In [47]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df1 = pd.read_html(html)[0]
df1.to_csv('beautifulsoup_pandas.csv',header=True,index=False)
print (df1.head())

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


#### Ignore cells with a borough that is Not assigned

In [48]:
index_1 = df1[df1['Borough']=='Not assigned'].index
df2 = df1.drop(index_1)
df3 = df2.rename(columns={"Postcode": "Postal Code","Neighbourhood": "Neighborhood"})
df3.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Not assigned neighborhood will be the same as the borough
#### Rename Postal Code and Neighborhood

In [49]:
df4=df3.replace(to_replace = 'Not assigned', value = df2['Borough'])
df4.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Combined into one row more than one neighborhood that can exist in the same postal code area

In [50]:
df5 = df4.groupby(['Postal Code','Borough'])['Neighborhood'].apply(list).reset_index(name='Neighborhood')
df5.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [51]:
df5.shape

(103, 3)

<br>

# PART2

#### Getting Latitude and longitude from postal code using .csv

In [52]:
df6 = pd.read_csv('Geospatial_Coordinates.csv')
df6.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Putting together the postal code, Borough, Neighborhood and respective Latitudes and Longitudes of each postal code

In [53]:
pd.merge(df5, df6, on="Postal Code")
neighborhoods = pd.merge(df5, df6, on="Postal Code")
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


#  __PART3__

#### Checking the number of boroughs and neighborhoods

In [54]:
print('{} boroughs and {} neighborhoods.'.format
      (len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]))

11 boroughs and 103 neighborhoods.


#### Putting back each neighborhoods in separeted lines

In [55]:
neighborhoods1 = pd.DataFrame([
    [p1,p2,p3,p4,p5] for p1,p2, P ,p4, p5 in neighborhoods.values
    for p3 in P 
], columns= neighborhoods.columns)
neighborhoods1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497


#### Dropping postal code

In [56]:
neighborhoods2 = neighborhoods1.drop(['Postal Code'], axis=1)
neighborhoods2.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Rouge,43.806686,-79.194353
1,Scarborough,Malvern,43.806686,-79.194353
2,Scarborough,Highland Creek,43.784535,-79.160497
3,Scarborough,Rouge Hill,43.784535,-79.160497
4,Scarborough,Port Union,43.784535,-79.160497


## Getting the coordinates using geolocator
#### Print the map of Toronto
#### Assigned each Neighborhood using latitude and longitude to the map

In [57]:
address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.653963, -79.387207.


In [58]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, Neighborhood in zip(neighborhoods2['Latitude'], neighborhoods2['Longitude'], neighborhoods2['Neighborhood']):
    label = '{}'.format(Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

In [59]:
CLIENT_ID = 'KVJKJIDIIU3IJNCHOORWH5FRNDM5BE51XRTDC1G2DOZYOBF0' # your Foursquare ID
CLIENT_SECRET = '4MQHADELHR1GE2PWPKAGNZTLOYYROFE55FPOM0LVGCCNZP0J' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: KVJKJIDIIU3IJNCHOORWH5FRNDM5BE51XRTDC1G2DOZYOBF0
CLIENT_SECRET:4MQHADELHR1GE2PWPKAGNZTLOYYROFE55FPOM0LVGCCNZP0J


## Start calling Four square API to get a venues list
#### Sign with client Id and cliente secret
#### Use the function to get venues information from each neighborhood
#### Filter .json and get - Venue name / Venue Latitude / Venue Longitude / Venue Category
#### The result is a dataframe called nearby_venues
#### Loop each neighborhood and get all the venues in Toronto
#### Total of 4516 rows

In [60]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0] ['name']

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [62]:
toronto_venues = getNearbyVenues(names=neighborhoods2['Neighborhood'],
                                   latitudes=neighborhoods2['Latitude'],
                                   longitudes=neighborhoods2['Longitude']
                                  )

Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview
Henry Farm
Oriole
Bayview Village
Silver Hills
York Mills
Newtonbrook
Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park
Don Mills South
Bathurst Manor
Downsview North
Wilson Heights
Northwood Park
York University
CFB Toronto
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens
Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West
Riverdale
The Beaches West
Indi

In [63]:
print(toronto_venues.shape)
toronto_venues.head()

(4469, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Rouge,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
3,Malvern,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
4,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


#### Get to know the number of venues per Neighborhoods

In [64]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Albion Gardens,9,9,9,9,9,9
Alderwood,8,8,8,8,8,8
Bathurst Manor,19,19,19,19,19,19
Bathurst Quay,14,14,14,14,14,14
Bayview Village,4,4,4,4,4,4
Beaumond Heights,9,9,9,9,9,9


In [65]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


#### Find the venue category and venue quantity exists in each Neighborhood

In [67]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [68]:
# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.drop('Neighborhood', 1).columns[:-1])
fixed_columns.insert(0,'Neighborhood')
th=toronto_onehot[fixed_columns]
th.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [69]:
th.shape

(4469, 275)

## Neighborhood Category
#### We need to find the percentage of each venue category per Neighborhood
#### How much each venue category represents per neighborhood
#### Getting the dataframe totonto_grouped

In [ ]:
toronto_grouped = th.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

#### Let´s find and organize the most 5 importants categories by its frequency in each neighborhood

In [29]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

#### Find sort and list the 10 most common venues using the Toronto_grouped dataframe
#### Put the result in neighborhoods_venues_sorted

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], 
                                                                          num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Bar,Restaurant,Burger Joint,American Restaurant,Hotel,Thai Restaurant,Cosmetics Shop
1,Agincourt,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,Agincourt North,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,Albion Gardens,Grocery Store,Pharmacy,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pizza Place,Comfort Food Restaurant,Comic Shop
4,Alderwood,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop


## Clustering Neighborhood
#### Start with cluster models using the toronto_grouped dataframe
#### Create an array that represents similar neighborhoods

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

#### Insert this array as a column in the data frame 

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,ClusterLabels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Adelaide,Coffee Shop,Café,Steakhouse,Bar,Restaurant,Burger Joint,American Restaurant,Hotel,Thai Restaurant,Cosmetics Shop
1,1,Agincourt,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,4,Agincourt North,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,1,Albion Gardens,Grocery Store,Pharmacy,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pizza Place,Comfort Food Restaurant,Comic Shop
4,1,Alderwood,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop


## Merge dataframes
#### Create two dataframes containing: Neighborhood coordenates and Cluster venues
#### I prefered passing data to those two dataframe before merging
#### bhh is created and will be used to print the cluster map considering Neighbourhood similarity

In [74]:
toronto_merged = pd.DataFrame(neighborhoods2)
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Rouge,43.806686,-79.194353
1,Scarborough,Malvern,43.806686,-79.194353
2,Scarborough,Highland Creek,43.784535,-79.160497
3,Scarborough,Rouge Hill,43.784535,-79.160497
4,Scarborough,Port Union,43.784535,-79.160497


In [75]:
ahh=neighborhoods_venues_sorted
ahh.head()

,ClusterLabels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Adelaide,Coffee Shop,Café,Steakhouse,Bar,Restaurant,Burger Joint,American Restaurant,Hotel,Thai Restaurant,Cosmetics Shop
1,1,Agincourt,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,4,Agincourt North,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,1,Albion Gardens,Grocery Store,Pharmacy,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pizza Place,Comfort Food Restaurant,Comic Shop
4,1,Alderwood,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop


In [76]:
bhh=pd.merge(toronto_merged, ahh, on='Neighborhood', how='inner')
bhh.head()

,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Rouge,43.806686,-79.194353,1,Print Shop,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
1,Scarborough,Malvern,43.806686,-79.194353,1,Print Shop,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,Scarborough,Highland Creek,43.784535,-79.160497,0,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
3,Scarborough,Rouge Hill,43.784535,-79.160497,0,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,Scarborough,Port Union,43.784535,-79.160497,0,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bhh['Latitude'], bhh['Longitude'], bhh['Neighborhood'], bhh['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
bhh.loc[bhh['ClusterLabels'] == 0, bhh.columns[[1] + list(range(5, bhh.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Highland Creek,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
3,Rouge Hill,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,Port Union,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
53,Northwood Park,Falafel Restaurant,Massage Studio,Coffee Shop,Caribbean Restaurant,Bar,Women's Store,Doner Restaurant,Diner,Discount Store,Dog Run
54,York University,Falafel Restaurant,Massage Studio,Coffee Shop,Caribbean Restaurant,Bar,Women's Store,Doner Restaurant,Diner,Discount Store,Dog Run


In [79]:
bhh.loc[bhh['ClusterLabels'] == 1, bhh.columns[[1] + list(range(5, bhh.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rouge,Print Shop,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
1,Malvern,Print Shop,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
5,Guildwood,Mexican Restaurant,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Women's Store,Dim Sum Restaurant,Diner
6,Morningside,Mexican Restaurant,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Women's Store,Dim Sum Restaurant,Diner
7,West Hill,Mexican Restaurant,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Women's Store,Dim Sum Restaurant,Diner
8,Woburn,Coffee Shop,Korean Restaurant,Convenience Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Cedarbrae,Fried Chicken Joint,Bakery,Hakka Restaurant,Lounge,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Dog Run,Diner
10,Scarborough Village,Grocery Store,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
11,East Birchmount Park,Discount Store,Coffee Shop,Convenience Store,Department Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
12,Ionview,Discount Store,Coffee Shop,Convenience Store,Department Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant


In [80]:
bhh.loc[bhh['ClusterLabels'] == 2, bhh.columns[[1] + list(range(5, bhh.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
180,Cloverdale,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
181,Islington,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
182,Martin Grove,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
183,Princess Gardens,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
184,West Deane Park,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop


In [81]:
bhh.loc[bhh['ClusterLabels'] == 3, bhh.columns[[1] + list(range(5, bhh.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downsview Central,Food Truck,Baseball Field,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
167,Humber Bay,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
168,King's Mill Park,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
169,Kingsway Park South East,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
170,Mimico NE,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
171,Old Mill South,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
172,The Queensway East,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
173,Royal York South East,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
174,Sunnylea,Construction & Landscaping,Baseball Field,Pool,Business Service,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
190,Emery,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop


In [82]:
bhh.loc[bhh['ClusterLabels'] == 4, bhh.columns[[1] + list(range(5, bhh.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Agincourt North,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
32,L'Amoreaux East,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
33,Milliken,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
34,Steeles East,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
44,York Mills West,Park,Bank,Electronics Store,Convenience Store,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
46,Parkwoods,BBQ Joint,Bus Stop,Park,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
55,CFB Toronto,Bus Stop,Airport,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
56,Downsview East,Bus Stop,Airport,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
67,East Toronto,Park,Convenience Store,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
73,Lawrence Park,Park,Swim School,Bus Line,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
